In [1]:
import cv2
import numpy as np

# Que 1

In [2]:
# Check if the polygon has exactly 4 vertices
# If it detects a rectangle or square than it draw the detected rectangle or square on the original image
# Display the centroid as a red circle.
# Write perimeter and centroid on the image.

def find_rectangle(image_path):
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Threshold the image to create a binary mask for white regions
    _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        epsilon = 0.04 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Check if the polygon has 4 vertices (a rectangle or square)
        if len(approx) == 4:
            perimeter = cv2.arcLength(approx, True)
            M = cv2.moments(approx)
            if M["m00"] != 0:
                centroid_x = int(M["m10"] / M["m00"])
                centroid_y = int(M["m01"] / M["m00"])
            else:
                centroid_x, centroid_y = 0, 0

            cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)
            cv2.circle(image, (centroid_x, centroid_y), 5, (0, 0, 255), -1)

            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(image, f"Perimeter: {perimeter:.2f}", (10, 30), font, 0.5, (0, 0, 255), 2)
            cv2.putText(image, f"Centroid: ({centroid_x}, {centroid_y})", (10, 60), font, 0.5, (0, 0, 255), 2)

            cv2.imshow("Rectangle or Square", image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            return True
    return False

image_path = 'data/rect1.jpg'

# Check if the figure contains a rectangle or square and display its parameters
if find_rectangle(image_path):
    print("The figure contains a rectangle or square.")
else:
    print("No rectangle or square found in the figure.")


The figure contains a rectangle or square.


# Que 2

In [3]:
# This function takes an image and determine the amount of darkness found in that image
# In this way it differentiates the two images whether the image is of a boy or a girl

def calculate_darkness_density(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _,binary = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)

    # Calculate the darkness density (number of dark pixels)
    darkness_density = np.sum(binary == 0) / np.prod(binary.shape)

    return darkness_density

image1_path = 'data/fig3.jpg'  
image2_path = 'data/fig4.jpg' 

darkness_density1 = calculate_darkness_density(image1_path)
darkness_density2 = calculate_darkness_density(image2_path)

# Determine which image has more darkness (more hair)
if darkness_density1 > darkness_density2:
    print("Image 1 has more darkness, likely a girl.")
else:
    print("Image 2 has more darkness, likely a girl.")
    

Image 2 has more darkness, likely a girl.


# Que 3

In [4]:
# This function takes the both images and calculate the variance of both images.
# On the basis of variance it differentiates the both images.
# The image with lower variance is considered the Blurred Image.

def is_blurred_image(original_image_path, blurred_image_path):
    
    original_image = cv2.imread(original_image_path, cv2.IMREAD_GRAYSCALE)
    blurred_image = cv2.imread(blurred_image_path, cv2.IMREAD_GRAYSCALE)

    # Calculate the variance of pixel values for both images
    variance_original = np.var(original_image)
    variance_blurred = np.var(blurred_image)

    # Determine which image is blurred based on variance
    if variance_blurred < variance_original:
        return "Blurred Image", "Original Image"
    else:
        return "Original Image", "Blurred Image"

original_image_path = 'data/fig5.jpg'
blurred_image_path = 'data/fig5_blur.jpg'

blurred_title, original_title = is_blurred_image(original_image_path, blurred_image_path)

original_image = cv2.imread(original_image_path)
blurred_image = cv2.imread(blurred_image_path)

cv2.imshow(blurred_title, blurred_image)
cv2.imshow(original_title, original_image)

cv2.waitKey(0)
cv2.destroyAllWindows()


# Que 4

In [5]:
# Each region is highlighted 
# A loop iterates through the contours, calculating the area and centroid of each region
# It than shows the centroid of each region on the boundry.

def calculate_and_display_areas(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresholded = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for i, contour in enumerate(contours, start=1):
        area = cv2.contourArea(contour)

        M = cv2.moments(contour)
        if M["m00"] != 0:
            centroid_x = int(M["m10"] / M["m00"])
            centroid_y = int(M["m01"] / M["m00"])
        else:
            centroid_x, centroid_y = 0, 0

        cv2.putText(image, f"Area {i}: {area:.2f} pixels", (centroid_x - 50, centroid_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv2.circle(image, (centroid_x, centroid_y), 5, (0, 0, 255), -1)

    cv2.imshow("Colored Bars with Areas", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

image_path = 'data/fig1.jpg'  
calculate_and_display_areas(image_path)


# Que 5

In [6]:
# First defining color ranges for each bar
# Check which color range the mean_color falls into
# Define the coordinates for the red arrow
# Calculate the area of red arrow
# calculating the area of each region and storing in array
# displaying the areas

def classify_color(mean_color):
    color_ranges = {
        'Yellow': ((200, 200, 0), (255, 255, 100)),
        'Light Gray': ((160, 160, 160), (190, 190, 190)),
        'Gray': ((100, 100, 100), (140, 140, 140)),
        'Dark Gray': ((50, 50, 50), (90, 90, 90))
    }
    
    for color, (lower, upper) in color_ranges.items():
        if np.all(mean_color >= lower) and np.all(mean_color <= upper):
            return color

    return 'Unknown'

def calculate_percentage_area_covered(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresholded = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    arrow_roi = (200, 50, 300, 150)  # (x, y, width, height)

    arrow_area = arrow_roi[2] * arrow_roi[3]
    bar_areas_covered = {}

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        roi_x, roi_y, roi_w, roi_h = arrow_roi

        x_intersection = max(0, min(x + w, roi_x + roi_w) - max(x, roi_x))
        y_intersection = max(0, min(y + h, roi_y + roi_h) - max(y, roi_y))

        intersection_area = x_intersection * y_intersection
        percentage_covered = (intersection_area / arrow_area) * 100
        mask = np.zeros_like(image, dtype=np.uint8)
        cv2.drawContours(mask, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)
        mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mean_color = cv2.mean(image, mask=mask_gray)[:3]
        bar_color = classify_color(mean_color)
        bar_areas_covered[bar_color] = percentage_covered

    print("Bar Area Covered(%)")
    for color, percentage in bar_areas_covered.items():
        print(f"{color}: {percentage:.0f}%")

image_path = 'data/fig2.jpg'  
calculate_percentage_area_covered(image_path)

Bar Area Covered(%)
Light Gray: 40%


# Que 6 

In [7]:
# First create a blank white canvas to draw each bone segment separately
# Extract the bounding rectangle of the bone segment
# Seperating each segment
# Calculating the height and width
# Marking the bone
# Displaying the bone

def segment_and_display_bones(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    canvas = np.zeros_like(image)
    for i, contour in enumerate(contours, start=1):
        
        x, y, w, h = cv2.boundingRect(contour)
        bone_segment = image[y:y+h, x:x+w]

        max_width = w
        max_height = h

        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(bone_segment, f"Max Width: {max_width}px", (10, 30), font, 1, (255, 255, 255), 2)
        cv2.putText(bone_segment, f"Max Height: {max_height}px", (10, 70), font, 1, (255, 255, 255), 2)

        cv2.imwrite(f"bone_segment_{i}.png", bone_segment)

        cv2.imshow(f"Bone Segment {i}", bone_segment)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

image_path = 'data/finger-bones.jpg'  

segment_and_display_bones(image_path)